In [ ]:
!pip install statsmodels

# Libraries

In [3]:
import numpy as np
import pandas as pd
 

#------------------------------------Plots
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import StandardScaler, MinMaxScaler


from collections import Counter
import re
from scipy.stats import chi2_contingency # Para la prueba de Chi-Cuadrado

#------------------------------------Plots Functions

import eda_utils
import importlib
importlib.reload(eda_utils)

from eda_utils import calculate_psi, plot_distribution_plotly, calculate_woe_iv
import utils_plot



spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
pd.set_option('display.max_columns', 500)


sns.set_style("whitegrid")
sns.set(rc = {'figure.figsize':(15,10)})



from statsmodels.stats.proportion import proportions_ztest 


# Load data 

In [ ]:
df = spark.sql("select * from data_credit_test_ds_chanllenges").toPandas()

df['d_vintage'] = pd.to_datetime(df['issue_d'], format='%b-%Y')

print('Dimension of the data:', df.shape)

Dimension of the data: (2260698, 151)


# Descriptive analysis

In [5]:
df['loan_status'] = df['loan_status'].fillna('NA')
df['loan_status'] = np.where(df['loan_status']=='Does not meet the credit policy. Status:Fully Paid', 'Fully Paid', df['loan_status'])
df['loan_status'] = np.where(df['loan_status']=='Does not meet the credit policy. Status:Charged Off', 'Charged Off', df['loan_status'])


#----------------------------------------------------------------------------------------------------------------------------------------------------------------------
#General definitions

df['target'] = np.where(df['loan_status']=='Fully Paid', 0.0, 
                        np.where(df['loan_status']=='Charged Off', 1.0, np.nan))

df_target = df[df['target'].notna()]

df_target['Set'] = np.where(df_target['d_vintage']>=pd.to_datetime('2016-07-01'), 'Test','Train')


In [7]:
df_group = df.groupby(['loan_status']).agg({'id':'count'}).reset_index()
df_group['pct_loan_status'] = (df_group['id']/df_group.id.sum()) *100

utils_plot.plot_chart(
    df= df_group.sort_values(by='pct_loan_status', ascending=False), 
    chart_type="bar",
    x_col="loan_status",
    y_col="pct_loan_status",
    text_col="",
    color_sequence= [  '#b4a7d6', ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Loan Status',
    y_label='%Booked Loans',
    title="Distribution of loan status for booked loans",
    width=1200,
    height=500,
)

Current Status (?)

In [8]:
df_current  = df[df['loan_status']=='Current']

df_current['year_vintage'] = df_current['d_vintage'].dt.year

df_group = df_current.groupby('year_vintage').agg({'id':'count'}).reset_index()

df_group['pct_target'] = (df_group['id'] / df_current.shape[0]) * 100

utils_plot.plot_chart(
    df= df_group, 
    chart_type="bar",
    x_col="year_vintage",
    y_col="pct_target",
    text_col="",
    color_sequence= [  '#b4a7d6', ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Booked Year',
    y_label='%Current Loans',
    title="Distribution of Current Loand by Booked Year",
    width=1200,
    height=500, 
)


## Target across the time to decide the Test set and the train

In [9]:
df_group = df.groupby(['d_vintage']).agg({'target': 'mean'}).reset_index()

df_group['target'] = df_group['target']*100

utils_plot.plot_chart(
    df= df_group, 
    chart_type="line",
    x_col="d_vintage",
    y_col="target",
    text_col="",
    color_sequence= [  '#b4a7d6', ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Booked Date',
    y_label='Mean of Charged Off Loans',
    title="Evolution of Charged Off Loans by Loan Booking Date",
    width=1200,
    height=500,
)

In [10]:
df_group = df_target.groupby(['d_vintage','Set']).agg({'id':'count'}).reset_index()

df_group['pct_target_pop'] = (df_group['id'] / df_target.shape[0]) * 100

utils_plot.plot_chart(
    df= df_group, 
    chart_type="bar",
    x_col="d_vintage",
    y_col="pct_target_pop",
    text_col="",
    color_sequence= [  '#b4a7d6',  '#6c6480'],
    text_as_percentage=True,
    group_col="Set",
    color ='Set',
    text_position="top center",
    x_label='Booked Year',
    y_label='%Current Loans',
    title="Distribution of Current Loand by Booked Year",
    width=1200,
    height=500, 
)

In [11]:
print('Training population distribution', df_group.groupby(['Set']).agg({'id':'sum'})/df_target.shape[0])

Training population distribution              id
Set            
Test   0.255053
Train  0.744947



## Distribution Analysis

### PSI

In [12]:


df_train =df[df['d_vintage']>=pd.to_datetime('2016-07-01')]
df_test =df[df['d_vintage']<pd.to_datetime('2016-07-01')]


df_train['verification_status_joint'] = df_train['verification_status_joint'].fillna('None')
df_test['verification_status_joint'] = df_test['verification_status_joint'].fillna('None')



columns_to_check_psi = df.columns.tolist()

psi_results = {}

print("Calculando PSI para las variables entre df_train y df_test:\n")

for column_name in columns_to_check_psi:
    if column_name not in df_train.columns or column_name not in df_test.columns:
        print(f"Advertencia: La columna '{column_name}' no se encuentra en ambos DataFrames. Omitiendo.")
        continue

    print(f"--- Variable: {column_name} ---")
    
    expected_series = df_train[column_name]
    actual_series = df_test[column_name]
    
    # Puedes especificar 'variable_type' si lo deseas, o dejar que la función lo infiera.
    # Si es una variable numérica con pocos valores únicos que quieres tratar como categórica,
    # o viceversa, especifícalo.
    # Por ejemplo, para una variable numérica:
    # psi_value, psi_df_detail = calculate_psi(expected_series, actual_series, bins=10, variable_type='numerical')
    # O para una categórica:
    # psi_value, psi_df_detail = calculate_psi(expected_series, actual_series, variable_type='categorical')
    
    # Dejando que la función infiera el tipo:
    psi_value, psi_df_detail = calculate_psi(expected_series, actual_series, bins=10) # bins se usa para numéricas
    
    psi_results[column_name] = {'psi_value': psi_value, 'psi_detail': psi_df_detail}
    
    if pd.isna(psi_value):
        print(f"PSI para '{column_name}': No se pudo calcular (NaN)")
    else:
        print(f"PSI para '{column_name}': {psi_value:.4f}")
        if psi_value < 0.1:
            print("Interpretación: Sin cambio significativo (Variable estable)")
        elif psi_value < 0.25:
            print("Interpretación: Cambio moderado (Requiere investigación)")
        else:
            print("Interpretación: Cambio significativo (Variable inestable)")
    
    # Opcional: mostrar el DataFrame de detalle para cada variable
    # print("\nDetalle del PSI:")
    # print(psi_df_detail)
    print("-" * 30 + "\n")

# Ahora psi_results contiene el PSI y el detalle para cada variable
# Puedes, por ejemplo, crear un resumen:
summary_psi = pd.DataFrame([(col, res['psi_value']) for col, res in psi_results.items()], 
                           columns=['Variable', 'PSI'])
summary_psi = summary_psi.sort_values(by='PSI', ascending=False)

print("\nResumen de PSI (ordenado de mayor a menor):")
print(summary_psi)

Calculando PSI para las variables entre df_train y df_test:

--- Variable: id ---
PSI para 'id': 49.9744
Interpretación: Cambio significativo (Variable inestable)
------------------------------

--- Variable: loan_amnt ---
PSI para 'loan_amnt': 0.0402
Interpretación: Sin cambio significativo (Variable estable)
------------------------------

--- Variable: funded_amnt ---
PSI para 'funded_amnt': 0.0404
Interpretación: Sin cambio significativo (Variable estable)
------------------------------

--- Variable: funded_amnt_inv ---
PSI para 'funded_amnt_inv': 0.0396
Interpretación: Sin cambio significativo (Variable estable)
------------------------------

--- Variable: term ---
PSI para 'term': 0.0004
Interpretación: Sin cambio significativo (Variable estable)
------------------------------

--- Variable: int_rate ---
PSI para 'int_rate': 0.0506
Interpretación: Sin cambio significativo (Variable estable)
------------------------------

--- Variable: installment ---
PSI para 'installment': 0.

In [13]:
list_total_psi = summary_psi[(summary_psi['PSI']>=0.25) | (summary_psi['PSI'].isna())]['Variable']

summary_psi[(summary_psi['PSI']>=0.25) | (summary_psi['PSI'].isna())].reset_index(drop=True)

,Variable,PSI
0,id,49.974388
1,url,49.974388
2,d_vintage,15.731325
3,issue_d,15.731325
4,emp_title,9.039951
5,desc,8.343615
6,last_pymnt_d,3.780655
7,out_prncp,2.053414
8,out_prncp_inv,2.052888
9,settlement_date,1.835869


In [14]:


variables_do_not_use_or_extreme_caution = {
    # Unique identifiers
    "id": "Unique loan identifier",  # PSI: 49.97
    "url": "URL of the loan page",  # PSI: 49.97

    # Free text with high cardinality
    "emp_title": "Job title of the borrower",  # PSI: 9.04
    "desc": "Loan purpose description provided by the borrower",  # PSI: 8.34
    "title": "Loan title provided by the borrower",  # PSI: 1.71

    # Variables with DATA LEAKAGE risk (post-origination information or consequence of default)
    "last_pymnt_d": "Date of the last payment received",  # PSI: 3.78
    "out_prncp": "Remaining outstanding principal balance",  # PSI: 2.05
    "out_prncp_inv": "Remaining outstanding principal balance (investor portion)",  # PSI: 2.05
    "total_pymnt": "Total payments received to date for the funded amount",  # PSI: 0.81
    "total_pymnt_inv": "Total payments received to date for the investor-funded portion",  # PSI: 0.80
    "total_rec_prncp": "Principal received to date",  # PSI: 0.81
    "total_rec_int": "Interest received to date",  # PSI: 0.26
    "loan_status": "Current loan status (e.g., Fully Paid, Charged Off)",  # PSI: 1.80 (If not target, it's leakage)

    # Variables related to post-default settlements or financial hardship
    "settlement_date": "Date a debt settlement plan was agreed upon",  # PSI: 1.84
    "debt_settlement_flag_date": "Date the debt settlement flag was activated",  # PSI: 1.20
    "settlement_status": "Status of a debt settlement plan (e.g., COMPLETE, ACTIVE)",  # PSI: 0.64
    "settlement_term": "Term (in months) of a debt settlement plan",  # PSI: 0.37
    "hardship_start_date": "Start date of a hardship plan",  # PSI: 0.97
    "payment_plan_start_date": "Start date of a payment plan (associated with hardship)",  # PSI: 0.95
    "hardship_end_date": "End date of a hardship plan",  # PSI: 0.90
    "hardship_payoff_balance_amount": "Payoff balance amount under a hardship plan"  # PSI: 0.27
}

#-------------------------------------------------------------------------------
# Category 2: REQUIRES INVESTIGATION or TRANSFORMATION (for high numerical PSI)
# (Possible transformation, or exclusion if not stabilized or not crucial)
#-------------------------------------------------------------------------------

variables_investigate_or_transform = {
    # Dates that may reflect temporary changes or need transformation (e.g., to age)
    "d_vintage": "Loan 'vintage' or original grouping date",  # PSI: 15.73
    "issue_d": "Month and year the loan was funded (issued)",  # PSI: 15.73
    "last_credit_pull_d": "Most recent month and year credit bureau was pulled for this loan",  # PSI: 1.31
    "earliest_cr_line": "Month and year of the borrower's earliest reported credit line",  # PSI: 0.37

    # Other categorical variables with distribution changes
    "verification_status_joint": "Income verification status for a joint application",  # PSI: 5.74
    "application_type": "Indicates if the application was individual or joint",  # PSI: 0.32
    "initial_list_status": "Initial listing status of the loan (for P2P platforms, e.g., 'w' for whole, 'f' for fractional)"  # PSI: 0.30
}

#-------------------------------------------------------------------------------
# Category 3: PSI is NaN - INVESTIGATE CAUSE and likely DO NOT USE
# (Stability could not be calculated, high risk if used without understanding why)
#-------------------------------------------------------------------------------
variables_psi_is_nan = {
    "revol_bal_joint": "Total revolving balance on joint accounts",  # PSI: NaN
    "sec_app_fico_range_low": "Lower bound of FICO range for the secondary applicant",  # PSI: NaN
    "sec_app_fico_range_high": "Upper bound of FICO range for the secondary applicant",  # PSI: NaN
    "sec_app_earliest_cr_line": "Earliest credit line date for the secondary applicant",  # PSI: NaN
    "sec_app_inq_last_6mths": "Credit inquiries in the last 6 months for the secondary applicant",  # PSI: NaN
    "sec_app_mort_acc": "Number of mortgage accounts for the secondary applicant",  # PSI: NaN
    "sec_app_open_acc": "Number of open credit lines for the secondary applicant",  # PSI: NaN
    "sec_app_revol_util": "Revolving credit utilization rate for the secondary applicant",  # PSI: NaN
    "sec_app_open_act_il": "Number of open installment accounts (excluding mortgages) for the secondary applicant",  # PSI: NaN
    "sec_app_num_rev_accts": "Number of revolving accounts for the secondary applicant",  # PSI: NaN
    "sec_app_chargeoff_within_12_mths": "Charge-offs within the last 12 months for the secondary applicant",  # PSI: NaN
    "sec_app_collections_12_mths_ex_med": "Collections in the last 12 months (excluding medical) for the secondary applicant",  # PSI: NaN
    "sec_app_mths_since_last_major_derog": "Months since last major derogatory event for the secondary applicant"  # PSI: NaN
}



psi_total_vars = list(variables_do_not_use_or_extreme_caution.keys()) + list(variables_investigate_or_transform.keys()) + list(variables_psi_is_nan.keys())

print('Number of features with high PSI:', len(psi_total_vars))


Number of features with high PSI: 41


### Deep  dive on High PSI features

In [15]:
date_features = [
    "d_vintage",
    "issue_d",
    "last_pymnt_d",
    "settlement_date",
    "last_credit_pull_d",
    "debt_settlement_flag_date",
    "hardship_start_date",
    "payment_plan_start_date",
    "hardship_end_date",
    "earliest_cr_line",
    "sec_app_earliest_cr_line"
]

for col in date_features:
   if col in df_train.columns and col in df_test.columns: # Good practice to check if column exists
        df_train[col] = pd.to_datetime(df_train[col], errors='coerce')
        df_test[col] = pd.to_datetime(df_test[col], errors='coerce')

/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/3340324263.py:17: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/3340324263.py:18: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/3340324263.py:17: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/3340324263.py:18: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`.

In [16]:
df_test[date_features].describe()

,d_vintage,issue_d,last_pymnt_d,settlement_date,last_credit_pull_d,debt_settlement_flag_date,hardship_start_date,payment_plan_start_date,hardship_end_date,earliest_cr_line,sec_app_earliest_cr_line
count,1119181,1119181,1118240,24598,1119130,24598,5449,5449,5449,1119152,0
mean,2014-11-13 23:06:45.029392128,2014-11-13 23:06:45.029392128,2017-01-10 01:50:23.247961600,2017-10-07 14:26:35.251645952,2018-04-16 14:25:55.495429632,2018-03-07 00:47:11.059435264,2017-11-20 12:48:29.598092032,2017-12-01 14:46:05.571664128,2018-02-01 23:16:39.596255232,1998-06-04 15:02:27.917709184,NaT
min,2007-06-01 00:00:00,2007-06-01 00:00:00,2007-12-01 00:00:00,2009-03-01 00:00:00,2007-05-01 00:00:00,2010-02-01 00:00:00,2017-01-01 00:00:00,2017-02-01 00:00:00,2017-03-01 00:00:00,1933-03-01 00:00:00,NaT
25%,2014-04-01 00:00:00,2014-04-01 00:00:00,2016-02-01 00:00:00,2017-05-01 00:00:00,2017-10-01 00:00:00,2017-11-01 00:00:00,2017-09-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00,1994-08-01 00:00:00,NaT
50%,2015-04-01 00:00:00,2015-04-01 00:00:00,2017-03-01 00:00:00,2017-12-01 00:00:00,2018-11-01 00:00:00,2018-06-01 00:00:00,2017-10-01 00:00:00,2017-10-01 00:00:00,2017-12-01 00:00:00,1999-11-01 00:00:00,NaT
75%,2015-11-01 00:00:00,2015-11-01 00:00:00,2018-05-01 00:00:00,2018-06-01 00:00:00,2019-03-01 00:00:00,2018-11-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-04-01 00:00:00,2003-09-01 00:00:00,NaT
max,2016-06-01 00:00:00,2016-06-01 00:00:00,2019-03-01 00:00:00,2019-03-01 00:00:00,2019-03-01 00:00:00,2019-03-01 00:00:00,2019-03-01 00:00:00,2019-04-01 00:00:00,2019-06-01 00:00:00,2013-05-01 00:00:00,NaT


#### Plots- Deep Dive

In [17]:
fig = utils_plot.plot_comparative_distribution_plotly(
    df_train,
    df_test,
    column_name='initial_list_status',
    plot_type='bar', 
    bins=30,        
    color_discrete_sequence=[  '#b4a7d6',  '#6c6480'],
    title_suffix=" - Loan Amount Distribution (Train vs Test)"
)

if fig:
    
    fig.update_layout(
        xaxis_title="Initial Status", 
        yaxis_title="Relative Proportion",      
        legend_title="Dataset" ,     
        title= 'Distribution comparisson of Initial Status (Train vs Test)'
    )


    fig.show()

In [18]:
fig = utils_plot.plot_comparative_distribution_plotly(
    df_train,
    df_test,
    column_name='application_type',
    plot_type='bar', 
    bins=30,        
    color_discrete_sequence=[  '#b4a7d6',  '#6c6480'],
    title_suffix=" - Loan Amount Distribution (Train vs Test)"
)

if fig:
    
    fig.update_layout(
        xaxis_title="Application Type", 
        yaxis_title="Relative Proportion",      
        legend_title="Dataset" ,     
        title= 'Distribution comparisson of Application Type (Train vs Test)'
    )


    fig.show()

In [19]:
fig = utils_plot.plot_comparative_distribution_plotly(
    df_train,
    df_test,
    column_name='verification_status_joint',
    plot_type='bar', 
    bins=30,        
    color_discrete_sequence=[  '#b4a7d6',  '#6c6480'],
    title_suffix=" - Verification Status (Train vs Test)"
)

if fig:
    
    fig.update_layout(
        xaxis_title="Verification Status", 
        yaxis_title="Relative Proportion",      
        legend_title="Dataset" ,     
        title= 'Distribution comparisson of Verification Status (Train vs Test)'
    )


    fig.show()

In [20]:


variables_do_not_use_or_extreme_caution = {
    # Unique identifiers
    "id": "Unique loan identifier",  # PSI: 49.97
    "url": "URL of the loan page",  # PSI: 49.97

    # Free text with high cardinality
    "emp_title": "Job title of the borrower",  # PSI: 9.04
    "desc": "Loan purpose description provided by the borrower",  # PSI: 8.34
    "title": "Loan title provided by the borrower",  # PSI: 1.71

    # Variables with DATA LEAKAGE risk (post-origination information or consequence of default)
    "last_pymnt_d": "Date of the last payment received",  # PSI: 3.78
    "out_prncp": "Remaining outstanding principal balance",  # PSI: 2.05
    "out_prncp_inv": "Remaining outstanding principal balance (investor portion)",  # PSI: 2.05
    "total_pymnt": "Total payments received to date for the funded amount",  # PSI: 0.81
    "total_pymnt_inv": "Total payments received to date for the investor-funded portion",  # PSI: 0.80
    "total_rec_prncp": "Principal received to date",  # PSI: 0.81
    "total_rec_int": "Interest received to date",  # PSI: 0.26
    "loan_status": "Current loan status (e.g., Fully Paid, Charged Off)",  # PSI: 1.80 (If not target, it's leakage)

    # Variables related to post-default settlements or financial hardship
    "settlement_date": "Date a debt settlement plan was agreed upon",  # PSI: 1.84
    "debt_settlement_flag_date": "Date the debt settlement flag was activated",  # PSI: 1.20
    "settlement_status": "Status of a debt settlement plan (e.g., COMPLETE, ACTIVE)",  # PSI: 0.64
    "settlement_term": "Term (in months) of a debt settlement plan",  # PSI: 0.37
    "hardship_start_date": "Start date of a hardship plan",  # PSI: 0.97
    "payment_plan_start_date": "Start date of a payment plan (associated with hardship)",  # PSI: 0.95
    "hardship_end_date": "End date of a hardship plan",  # PSI: 0.90
    "hardship_payoff_balance_amount": "Payoff balance amount under a hardship plan"  # PSI: 0.27
}

In [21]:
post_default_vars_psi = [
    "settlement_date",
    "debt_settlement_flag_date",
    "settlement_status",
    "settlement_term",
    "hardship_start_date",
    "payment_plan_start_date",
    "hardship_end_date",
    "hardship_payoff_balance_amount"
]
#-----------------------------------------------------------------------------------------------------------------------
leakage_vars_psi = [
    "last_pymnt_d",
    "out_prncp",
    "out_prncp_inv",
    "total_pymnt",
    "total_pymnt_inv",
    "total_rec_prncp",
    "total_rec_int",
    "loan_status"
]

#------------------------------------------------------------------------------------------------------------------------
features_to_drop_deep_psi = ['verification_status_joint', 'application_type', 'url',     "last_pymnt_d", "settlement_date",  "last_credit_pull_d", 
                             "debt_settlement_flag_date", "hardship_start_date", "payment_plan_start_date", "hardship_end_date"]

In [22]:
len(list(variables_psi_is_nan.keys())+ features_to_drop_deep_psi+features_to_drop_deep_psi +leakage_vars_psi)

41

# Feature distribution analysis

In [45]:
df_model = df_target.drop(list(variables_psi_is_nan.keys())+ features_to_drop_deep_psi + post_default_vars_psi +leakage_vars_psi, axis=1)

print('Initial dimension of data model:', df_model.shape)

#Dates transformation

date_features_refined = [
    "issue_d",
    "earliest_cr_line",
    "last_pymnt_d",
    "next_pymnt_d",
    "last_credit_pull_d",
    "hardship_start_date",
    "hardship_end_date",
    "payment_plan_start_date",
    "debt_settlement_flag_date",
    "settlement_date",
    "d_vintage"
]

for col in date_features_refined:
    if col in df_model.columns:
        print(f"  Converting '{col}'...")
        df_model[col] = pd.to_datetime(df_model[col], errors='coerce')
    else:
        print(f"  Warning: Column '{col}' not found in df_model.")


Initial dimension of data model: (1348059, 120)
  Converting 'issue_d'...
  Converting 'earliest_cr_line'...


/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/4191784565.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/4191784565.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

/var/folders/v4/7dg4p38j5c31bxdhb2vhmmb00000gp/T/ipykernel_73975/4191784565.py:24: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



  Converting 'next_pymnt_d'...
  Converting 'd_vintage'...


### Null Values & Detecting constant features

In [46]:
print('Dimensiones del DataFrame:', df_model.shape)

variable_quality_summary = eda_utils.assess_variable_quality(
    df=df_model,
    null_perc_threshold=95.0,  
    constant_perc_threshold=85.0 
)

Dimensiones del DataFrame: (1348059, 120)


In [47]:
df_group = variable_quality_summary.groupby('Recommendation').agg({'Variable':'count'}).reset_index()

df_group['pct_features'] = (df_group['Variable'] / df_group['Variable'].sum()) * 100

utils_plot.plot_chart(
    df= df_group.sort_values(by='pct_features', ascending=False), 
    chart_type="bar",
    x_col="Recommendation",
    y_col="pct_features",
    text_col="",
    color_sequence= [  '#b4a7d6', ],
    text_as_percentage=True,
    text_position="top center",
    x_label='Feature',
    y_label='%Features',
    title="Distribution of features by recommendation from Null values and Variability analysis",
    width=1200,
    height=500,
)

### Removing problematic features

In [48]:
remove_high_nulls_constant_like = list(variable_quality_summary[variable_quality_summary['Recommendation']=='Remove: High Nulls & Constant-Like']['Variable'].values)

remove_constant_like = list(variable_quality_summary[variable_quality_summary['Recommendation']=='Remove: Constant (Single Value)']['Variable'].values)

remove_high_nulls = list(variable_quality_summary[variable_quality_summary['Recommendation']=='Review: High Nulls']['Variable'].values)

remove_constant_like_prob = list(variable_quality_summary[variable_quality_summary['Recommendation']=='Review: Constant-Like']['Variable'].values)

remove_total = list(np.unique(np.array(remove_high_nulls_constant_like + remove_constant_like + remove_high_nulls + remove_constant_like_prob)))

print('Number of variables to remove:', len(remove_total))


df_model = df_model.drop(remove_total, axis=1)

print('New dimension of dataset',df_model.shape)


Number of variables to remove: 32
New dimension of dataset (1348059, 88)


#### Null analysis across the time 

In [49]:
vintage_null_alerts_df = eda_utils.generate_null_alerts_by_vintage(
        df=df_model,
        group_col='d_vintage',
        analyze_cols=df_model.columns,
        high_null_perc_threshold=90,
        min_increase_perc_threshold=15
    )

Advertencia: La columna de agrupación 'd_vintage' no debe estar en `analyze_cols` y será omitida.


In [50]:
print('Number of problematic features', len(vintage_null_alerts_df[vintage_null_alerts_df['Alert_Type']=='High Null Percentage']['Variable'].value_counts()))

Number of problematic features 50


array(['desc', 'mths_since_last_record', 'mths_since_last_major_derog',
       'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
       'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
       'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
       'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
       'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
       'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
       'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
   

In [51]:
try:
    alternative_purple_scale = [px.colors.sequential.Purples[2], px.colors.sequential.Purples[5]]

except IndexError:
    # Fallback si la escala de Purples es más corta de lo esperado (poco probable)
    alternative_purple_scale = [px.colors.sequential.Purples[0], px.colors.sequential.Purples[-1]]



chosen_color_sequence = alternative_purple_scale


df_analisis_nulos_por_vintage = eda_utils.analyze_null_values_by_group(
        df=df_model,  
        group_col='d_vintage',  
        analyze_cols=None 
                          
    )





 #------------------------------------------------------------------------------------------------------------------------   

# --- Configuración Inicial ---

# Lista de variables a analizar (ya definida en tu script)
variables_con_altos_nulos = list(vintage_null_alerts_df[vintage_null_alerts_df['Alert_Type']=='High Null Percentage']['Variable'].unique())

# Variable objetivo y colores para gráficos
target_variable_name = 'target'
main_plot_color = '#C9C0E0'

# --- 1. Verificar el Porcentaje de Nulos Global ---
print("--- 1. Porcentaje de Nulos Global ---")
global_null_percentages = df_model[variables_con_altos_nulos].isnull().mean() * 100
print(global_null_percentages.sort_values(ascending=False))

# --- 2. Analizar la Distribución de Nulos a Través de los Vintages ---
print("\n--- 2. Distribución de Nulos por Vintage (Análisis y Gráficos) ---")
try:
    df_analisis_nulos_por_vintage = analyze_null_values_by_group(
        df=df_model,
        group_col='d_vintage',
        analyze_cols=variables_con_altos_nulos
    )
    
    variables_a_graficar = global_null_percentages.sort_values(ascending=False).head(5).index # Graficar el top 5 para no saturar
    print(f"Mostrando gráficos de nulos por vintage para el Top {len(variables_a_graficar)} de variables con más nulos...")

    for var in variables_a_graficar:
        col_name = (var, 'Null Percentage (%)')
        if col_name in df_analisis_nulos_por_vintage.columns:
            vintage_col_name = df_analisis_nulos_por_vintage.index.name or 'Vintage'
            fig = px.line(
                x=df_analisis_nulos_por_vintage.index,
                y=df_analisis_nulos_por_vintage[col_name],
                title=f'Porcentaje de Nulos para "{var}" por Vintage',
                labels={'x': vintage_col_name, 'y': 'Null Percentage (%)'},
                markers=True
            )
            fig.update_traces(line=dict(color=main_plot_color))
            fig.update_layout(template='plotly_white')
            fig.show()
except Exception as e:
    print(f"Error al analizar nulos por vintage: {e}. Omite el paso 2.")

# --- 3. Entender el Significado de la Variable (Dominio del Problema) ---
print("\n--- 3. Entender el Significado (Conceptual) ---")
print("Este paso es conceptual. Analiza qué significa que una variable como 'mths_since_last_record' sea nula. ¿Significa que nunca hubo un registro, o que el dato no se capturó?")

# --- 4. Relación de Nulidad con la Variable Objetivo (Análisis Estadístico Completo) ---
print("\n--- 4. Relación de Nulidad con la Variable Objetivo (Resumen Tabular) ---")
all_vars_summary_data = []

for var in variables_con_altos_nulos:
    # Usamos un bloque try-except general por variable para aislar errores
    try:
        if var not in df_model.columns:
            print(f"Advertencia: La variable '{var}' no se encontró. Omitiendo.")
            continue

        # Columna temporal para análisis
        null_indicator_col = f'{var}_is_null'
        df_model[null_indicator_col] = df_model[var].isnull().map({True: 'Null', False: 'Not Null'})
        
        # a. Estadísticas Descriptivas
        summary_stats = df_model.groupby(null_indicator_col)[target_variable_name].agg(['mean', 'count', 'std'])
        mean_null = summary_stats.loc['Null', 'mean'] if 'Null' in summary_stats.index else 0
        count_null = summary_stats.loc['Null', 'count'] if 'Null' in summary_stats.index else 0
        
        mean_not_null = summary_stats.loc['Not Null', 'mean'] if 'Not Null' in summary_stats.index else 0
        count_not_null = summary_stats.loc['Not Null', 'count'] if 'Not Null' in summary_stats.index else 0

        # b. Information Value (IV)
        iv_score, iv_interp = np.nan, "No calculable"
        if count_null > 0 and count_not_null > 0:
            _, iv_score = calculate_woe_iv(df_model, null_indicator_col, target_variable_name)
            if iv_score < 0.02: iv_interp = 'Inútil'
            elif iv_score < 0.1: iv_interp = 'Débil'
            elif iv_score < 0.3: iv_interp = 'Medio'
            elif iv_score < 0.5: iv_interp = 'Fuerte'
            else: iv_interp = 'Sospechoso'

        # c. Tabla de Contingencia (base para Chi2 y Z-test)
        contingency_table = pd.crosstab(df_model[null_indicator_col], df_model[target_variable_name])
        
        # d. Test Chi-cuadrado
        chi2_stat, chi2_p, chi2_interp = np.nan, np.nan, "No aplicable"
        if contingency_table.shape == (2, 2): # Solo si hay nulos/no-nulos y target 0/1
            chi2_stat, chi2_p, _, _ = chi2_contingency(contingency_table)
            chi2_interp = 'Significativo (p<0.05)' if chi2_p < 0.05 else 'No Significativo'

        # e. Z-test para Proporciones (¡NUEVA MEJORA!)
        z_stat, z_p, z_interp = np.nan, np.nan, "No aplicable"
        if contingency_table.shape == (2, 2):
            counts = np.array([contingency_table.loc['Null', 1], contingency_table.loc['Not Null', 1]])
            nobs = np.array([count_null, count_not_null])
            z_stat, z_p = proportions_ztest(counts, nobs, value=0)
            z_interp = 'Diferencia Significativa (p<0.05)' if z_p < 0.05 else 'Sin Diferencia Significativa'

        all_vars_summary_data.append({
            'Variable': var,
            'Target_Mean_if_Null': mean_null,
            'Target_Mean_if_Not_Null': mean_not_null,
            'IV_Score_Nullity': iv_score,
            'IV_Interpretation': iv_interp,
            'Z_Test_Interpretation': z_interp,
            'Z_P_Value': z_p,
            'Chi2_Interpretation': chi2_interp,
            'Chi2_P_Value': chi2_p,
            'Count_if_Null': count_null,
            'Count_if_Not_Null': count_not_null,
        })
    except Exception as e:
        print(f"Error procesando la variable '{var}': {e}. Omitiendo.")
    finally:
        # Limpiar columna temporal de forma segura
        if null_indicator_col in df_model.columns:
            df_model.drop(columns=[null_indicator_col], inplace=True)

# Crear y mostrar el DataFrame de resumen final
if all_vars_summary_data:
    df_target_analysis_summary = pd.DataFrame(all_vars_summary_data)
    print("\nResumen del Análisis de Nulidad vs Target:")
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1200)
    print(df_target_analysis_summary.sort_values(by='IV_Score_Nullity', ascending=False))

# --- 5. Correlación de Nulidad entre Variables ---
print("\n--- 5. Correlación de Nulidad entre Variables (Heatmap) ---")
try:
    df_null_indicators = df_model[variables_con_altos_nulos].isnull().astype(int)
    null_correlation_matrix = df_null_indicators.corr()
    fig_heatmap = px.imshow(
        null_correlation_matrix,
        title='Missingness Correlation Heatmap',
        color_continuous_scale='PuOr_r',
        text_auto=False
    )
    fig_heatmap.update_layout(template='plotly_white', width=900, height=800)
    fig_heatmap.show()
except Exception as e:
    print(f"No se pudo generar el heatmap de correlación: {e}")

# --- Pasos Conceptuales Finales ---
print("\n--- 6. Impacto de Imputación vs. Eliminación (Conceptual) ---")
print("Basado en el IV y el Z-test, si la nulidad es un 'Predictor Fuerte' y la diferencia es 'Significativa', considera crear una categoría 'Missing' en lugar de imputar.")

print("\n--- 7. Fuente de Datos y Periodo (Conceptual) ---")
print("El análisis por vintage puede revelar si los nulos son un problema reciente (ej. cambio en un formulario) o si siempre han existido.")

print("\n--- Fin de la Exploración ---")

--- 1. Porcentaje de Nulos Global ---
desc                              90.656566
mths_since_last_record            82.981828
mths_since_recent_bc_dlq          76.334864
mths_since_last_major_derog       73.758567
mths_since_recent_revol_delinq    66.621491
il_util                           65.504774
mths_since_rcnt_il                61.175141
all_util                          60.124520
inq_last_12m                      60.120662
open_acc_6m                       60.120662
total_cu_tl                       60.120662
max_bal_bc                        60.120588
inq_fi                            60.120588
open_rv_12m                       60.120588
open_rv_24m                       60.120588
total_bal_il                      60.120588
open_il_24m                       60.120588
open_il_12m                       60.120588
open_act_il                       60.120588
mths_since_recent_inq             13.116637
mo_sin_old_il_acct                 8.035553
pct_tl_nvr_dlq                     5.2


--- 3. Entender el Significado (Conceptual) ---
Este paso es conceptual. Analiza qué significa que una variable como 'mths_since_last_record' sea nula. ¿Significa que nunca hubo un registro, o que el dato no se capturó?

--- 4. Relación de Nulidad con la Variable Objetivo (Resumen Tabular) ---

Resumen del Análisis de Nulidad vs Target:
                          Variable  Target_Mean_if_Null  Target_Mean_if_Not_Null  IV_Score_Nullity IV_Interpretation              Z_Test_Interpretation      Z_P_Value     Chi2_Interpretation   Chi2_P_Value  Count_if_Null  Count_if_Not_Null
31           mths_since_recent_inq             0.150780                 0.207181          0.015539            Inútil  Diferencia Significativa (p<0.05)   0.000000e+00  Significativo (p<0.05)   0.000000e+00         176820            1171239
15                        all_util             0.184119                 0.223403          0.014300            Inútil  Diferencia Significativa (p<0.05)   0.000000e+00  Significativ


--- 6. Impacto de Imputación vs. Eliminación (Conceptual) ---
Basado en el IV y el Z-test, si la nulidad es un 'Predictor Fuerte' y la diferencia es 'Significativa', considera crear una categoría 'Missing' en lugar de imputar.

--- 7. Fuente de Datos y Periodo (Conceptual) ---
El análisis por vintage puede revelar si los nulos son un problema reciente (ej. cambio en un formulario) o si siempre han existido.

--- Fin de la Exploración ---


In [ ]:


# --- Configuración ---
TARGET = 'target'

# Lista de variables a analizar. 
# He seleccionado un mix representativo. ¡Puedes ampliar esta lista con todas tus variables!
variables_to_analyze = [
    # Variables categóricas clave
    'grade',
    'sub_grade',
    'home_ownership',
    'verification_status',
    'purpose',
    'term',
    # Variables numéricas clave
    'loan_amnt',
    'int_rate',
    'installment',
    'annual_inc',
    'dti',
    'open_acc',
    'pub_rec',
    'revol_bal',
    'revol_util',
    'total_acc',
    'mths_since_last_delinq', # Variable con nulos que podría ser predictiva
    'mths_since_recent_inq'   # Otra variable con nulos
]

# --- Bucle de Análisis de IV ---
iv_results = []
print(f"Calculando Information Value para {len(variables_to_analyze)} variables...")

for var in variables_to_analyze:
    print(f"Procesando: {var}...")
    try:
        # La función devuelve el df detallado y el IV total. Solo necesitamos el IV total.
        df_woe_iv, iv_score = calculate_woe_iv(df_model, var, TARGET)
        
        iv_results.append({
            'Variable': var,
            'IV_Score': iv_score
        })
        
    except Exception as e:
        print(f"  -> ¡Error! No se pudo calcular el IV para '{var}'. Razón: {e}")
        iv_results.append({
            'Variable': var,
            'IV_Score': np.nan
        })

print("\nCálculo de IV completado.")

# --- Crear y Mostrar el DataFrame de Resultados ---
if iv_results:
    # Convertir lista de resultados a DataFrame
    df_iv_summary = pd.DataFrame(iv_results).sort_values(by='IV_Score', ascending=False).reset_index(drop=True)

    # Añadir interpretación basada en las reglas de Siddiqi
    def iv_interpretation(iv):
        if pd.isna(iv):
            return "Error en cálculo"
        if iv < 0.02: return 'Inútil'
        elif iv < 0.1: return 'Predictor Débil'
        elif iv < 0.3: return 'Predictor Medio'
        elif iv < 0.5: return 'Predictor Fuerte'
        else: return 'Sospechoso'

    df_iv_summary['Interpretation'] = df_iv_summary['IV_Score'].apply(iv_interpretation)
    
    print("Ranking de Poder Predictivo de las Variables (Information Value):")
    display(df_iv_summary)
else:
    print("No se pudieron generar resultados de IV.")

# --- Ejemplo para ver el detalle de una variable FUERTE ---
# Una vez que tengas el ranking, puedes inspeccionar las variables más fuertes.
# Por ejemplo, si 'grade' sale con IV alto, puedes hacer esto:
print("\nEjemplo de tabla de detalle para la variable 'grade':")
try:
    df_detail_grade, _ = calculate_woe_iv(df_model, 'grade', TARGET)
    display(df_detail_grade)
except Exception as e:
    print(f"No se pudo generar la tabla de detalle para 'grade': {e}")

In [53]:
df_target_analysis_summary

,Variable,Target_Mean_if_Null,Target_Mean_if_Not_Null,IV_Score_Nullity,IV_Interpretation,Z_Test_Interpretation,Z_P_Value,Chi2_Interpretation,Chi2_P_Value,Count_if_Null,Count_if_Not_Null
0,desc,0.204329,0.155683,0.008538,Inútil,Diferencia Significativa (p<0.05),0.000000e+00,Significativo (p<0.05),0.000000e+00,1222104,125955
1,mths_since_last_record,0.194039,0.227792,0.006057,Inútil,Diferencia Significativa (p<0.05),5.566851e-297,Significativo (p<0.05),6.187104e-297,1118644,229415
2,mths_since_last_major_derog,0.192800,0.219412,0.005246,Inútil,Diferencia Significativa (p<0.05),2.438958e-253,Significativo (p<0.05),2.650821e-253,994309,353750
3,tot_coll_amt,0.157650,0.202101,0.004158,Inútil,Diferencia Significativa (p<0.05),4.669666e-181,Significativo (p<0.05),5.366986e-181,70276,1277783
4,tot_cur_bal,0.157650,0.202101,0.004158,Inútil,Diferencia Significativa (p<0.05),4.669666e-181,Significativo (p<0.05),5.366986e-181,70276,1277783
5,open_acc_6m,0.184120,0.223398,0.014297,Inútil,Diferencia Significativa (p<0.05),0.000000e+00,Significativo (p<0.05),0.000000e+00,810462,537597
6,open_act_il,0.184120,0.223397,0.014296,Inútil,Diferencia Significativa (p<0.05),0.000000e+00,Significativo (p<0.05),0.000000e+00,810461,537598
7,open_il_12m,0.184120,0.223397,0.014296,Inútil,Diferencia Significativa (p<0.05),0.000000e+00,Significativo (p<0.05),0.000000e+00,810461,537598
8,open_il_24m,0.184120,0.223397,0.014296,Inútil,Diferencia Significativa (p<0.05),0.000000e+00,Significativo (p<0.05),0.000000e+00,810461,537598
9,mths_since_rcnt_il,0.185531,0.222241,0.012362,Inútil,Diferencia Significativa (p<0.05),0.000000e+00,Significativo (p<0.05),0.000000e+00,824677,523382


Evaluating general predictive power of the features

In [54]:


# --- Configuración ---
TARGET = 'target'

# Lista de variables a analizar (proporcionada por ti)
variables_to_analyze = [
    'desc', 'mths_since_last_record', 'mths_since_last_major_derog',
    'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il',
    'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
    'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
    'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
    'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
    'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op',
    'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',
    'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
    'mths_since_recent_inq', 'mths_since_recent_revol_delinq',
    'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl',
    'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl',
    'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats',
    'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75',
    'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
    'total_il_high_credit_limit'
]

# --- Bucle de Análisis de IV ---
iv_results = []
print(f"Calculando Information Value para {len(variables_to_analyze)} variables...")

for var in variables_to_analyze:
    print(f"Procesando: {var}...")
    try:
        # La función devuelve el df detallado y el IV total. Solo necesitamos el IV total.
        df_woe_iv, iv_score = calculate_woe_iv(df_model, var, TARGET)
        
        iv_results.append({
            'Variable': var,
            'IV_Score': iv_score
        })
        
    except Exception as e:
        print(f"  -> ¡Error! No se pudo calcular el IV para '{var}'. Razón: {e}")
        iv_results.append({
            'Variable': var,
            'IV_Score': np.nan
        })

print("\nCálculo de IV completado.")

# --- Crear y Mostrar el DataFrame de Resultados ---
if iv_results:
    # Convertir lista de resultados a DataFrame
    df_iv_summary = pd.DataFrame(iv_results).sort_values(by='IV_Score', ascending=False).reset_index(drop=True)

    # Añadir interpretación basada en las reglas de Siddiqi
    def iv_interpretation(iv):
        if pd.isna(iv):
            return "Error en cálculo"
        if iv < 0.02: return 'Inútil'
        elif iv < 0.1: return 'Predictor Débil'
        elif iv < 0.3: return 'Predictor Medio'
        elif iv < 0.5: return 'Predictor Fuerte'
        else: return 'Sospechoso'

    df_iv_summary['Interpretation'] = df_iv_summary['IV_Score'].apply(iv_interpretation)
    
    print("Ranking de Poder Predictivo de las Variables (Information Value):")
    display(df_iv_summary)
else:
    print("No se pudieron generar resultados de IV.")

Calculando Information Value para 50 variables...
Procesando: desc...
Procesando: mths_since_last_record...
Procesando: mths_since_last_major_derog...
Procesando: tot_coll_amt...
Procesando: tot_cur_bal...
Procesando: open_acc_6m...
Procesando: open_act_il...
Procesando: open_il_12m...
Procesando: open_il_24m...
Procesando: mths_since_rcnt_il...
Procesando: total_bal_il...
Procesando: il_util...
Procesando: open_rv_12m...
Procesando: open_rv_24m...
Procesando: max_bal_bc...
Procesando: all_util...
Procesando: total_rev_hi_lim...
Procesando: inq_fi...
Procesando: total_cu_tl...
Procesando: inq_last_12m...
Procesando: acc_open_past_24mths...
Procesando: avg_cur_bal...
Procesando: bc_open_to_buy...
Procesando: bc_util...
Procesando: mo_sin_old_il_acct...
Procesando: mo_sin_old_rev_tl_op...
Procesando: mo_sin_rcnt_rev_tl_op...
Procesando: mo_sin_rcnt_tl...
Procesando: mort_acc...
Procesando: mths_since_recent_bc...
Procesando: mths_since_recent_bc_dlq...
Procesando: mths_since_recent_inq..

,Variable,IV_Score,Interpretation
0,desc,1.916766,Sospechoso
1,acc_open_past_24mths,0.064334,Predictor Débil
2,bc_open_to_buy,0.057767,Predictor Débil
3,avg_cur_bal,0.052992,Predictor Débil
4,tot_hi_cred_lim,0.048874,Predictor Débil
5,num_tl_op_past_12m,0.045755,Predictor Débil
6,tot_cur_bal,0.042884,Predictor Débil
7,total_bc_limit,0.041359,Predictor Débil
8,mort_acc,0.039339,Predictor Débil
9,all_util,0.035598,Predictor Débil


In [55]:
features_to_drop = ['desc', 'mths_since_last_record', ] + df_iv_summary[df_iv_summary['Interpretation']=='Inútil']['Variable'].tolist()

df_model = df_model.drop(features_to_drop, axis=1)

In [58]:
####Savind data until here
df_model.reset_index(drop=True).to_csv("df_model.csv", index=False)

## Correlation analysis + IV

In [31]:


df_model = pd.read_csv('df_model.csv')
print('Dimension of data', df_model.shape)

####### Quick data transformation

# Create datetime columns
df_model['issue_d_dt'] = pd.to_datetime(df_model['issue_d'], errors='coerce')
df_model['earliest_cr_line_dt'] = pd.to_datetime(df_model['earliest_cr_line'], errors='coerce')

# Calculate difference and create new feature
time_difference = df_model['issue_d_dt'] - df_model['earliest_cr_line_dt']
df_model['credit_history_length_months'] = (time_difference.dt.days / 30.44).round(2)

# --- CORRECCIÓN AQUÍ ---
# Se corrige el error de tipeo y se eliminan también las columnas de fecha originales
# que ya no son necesarias.
df_model = df_model.drop(columns=['issue_d', 'earliest_cr_line', 'issue_d_dt', 'earliest_cr_line_dt'])

df_model.head()

Dimension of data (1348059, 61)


,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,purpose,title,zip_code,addr_state,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,tot_cur_bal,open_acc_6m,open_rv_12m,open_rv_24m,all_util,total_rev_hi_lim,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_actv_rev_tl,num_rev_tl_bal_gt_0,num_tl_op_past_12m,percent_bc_gt_75,tot_hi_cred_lim,total_bc_limit,d_vintage,target,Set,credit_history_length_months
0,68407277,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,MORTGAGE,55000.0,Not Verified,debt_consolidation,Debt consolidation,190xx,PA,5.91,0.0,675.0,679.0,1.0,30.0,7.0,0.0,2765.0,29.7,13.0,w,122.67,564.0,560.0,144904.0,2.0,3.0,3.0,34.0,9300.0,4.0,4.0,20701.0,1506.0,37.2,128.0,3.0,3.0,1.0,4.0,4.0,4.0,4.0,3.0,0.0,178050.0,2400.0,2015-12-01,0.0,Train,148.00
1,68355089,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,MORTGAGE,65000.0,Not Verified,small_business,Business,577xx,SD,16.06,1.0,715.0,719.0,4.0,6.0,22.0,0.0,21470.0,19.2,38.0,w,926.35,699.0,695.0,204396.0,1.0,2.0,3.0,29.0,111800.0,6.0,4.0,9733.0,57830.0,27.1,192.0,2.0,2.0,4.0,2.0,0.0,5.0,5.0,2.0,7.7,314017.0,79300.0,2015-12-01,0.0,Train,191.98
2,68341763,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,truck driver,10+ years,MORTGAGE,63000.0,Not Verified,home_improvement,NaN,605xx,IL,10.78,0.0,695.0,699.0,0.0,NaN,6.0,0.0,7869.0,56.2,18.0,w,15813.30,704.0,700.0,189699.0,0.0,0.0,2.0,65.0,14000.0,1.0,6.0,31617.0,2737.0,55.9,184.0,14.0,14.0,5.0,101.0,10.0,3.0,3.0,0.0,50.0,218418.0,6200.0,2015-12-01,0.0,Train,183.97
3,68476807,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,MORTGAGE,104433.0,Source Verified,major_purchase,Major purchase,174xx,PA,25.37,1.0,695.0,699.0,3.0,12.0,12.0,0.0,21929.0,64.5,35.0,w,10128.96,704.0,700.0,331730.0,1.0,4.0,7.0,78.0,34000.0,3.0,10.0,27644.0,4567.0,77.5,210.0,4.0,4.0,6.0,4.0,1.0,6.0,6.0,4.0,60.0,439570.0,20300.0,2015-12-01,0.0,Train,209.99
4,68426831,11950.0,11950.0,11950.0,36 months,13.44,405.18,C,C3,Veterinary Tecnician,4 years,RENT,34000.0,Source Verified,debt_consolidation,Debt consolidation,300xx,GA,10.20,0.0,690.0,694.0,0.0,NaN,5.0,0.0,8822.0,68.4,6.0,w,7653.56,759.0,755.0,12798.0,0.0,0.0,0.0,76.0,12900.0,0.0,0.0,2560.0,844.0,91.0,54.0,32.0,32.0,0.0,36.0,NaN,3.0,3.0,0.0,100.0,16900.0,9400.0,2015-12-01,0.0,Train,337.98


In [ ]:
# --- Configuración ---
TARGET = 'target'
CORRELATION_THRESHOLD = 0.70

# --- 1. Definir la Lista Completa de Variables a Analizar ---
all_variables_to_analyze = list(set(df_model.columns))
vars_to_remove = ['issue_d', 'id', 'd_vintage', 'earliest_cr_line', TARGET]
for var in vars_to_remove:
    if var in all_variables_to_analyze:
        all_variables_to_analyze.remove(var)

print(f"Se analizará el IV para un total de {len(all_variables_to_analyze)} variables únicas.")

# --- 2. Recalcular IV para todas las variables ---
iv_results = []
for var in all_variables_to_analyze:
    try:
        _, iv_score = calculate_woe_iv(df_model, var, TARGET)
        iv_results.append({'Variable': var, 'IV_Score': iv_score})
    except Exception as e:
        print(f"  -> Advertencia: No se pudo calcular el IV para '{var}'. Razón: {e}")
        iv_results.append({'Variable': var, 'IV_Score': np.nan})

df_iv_all = pd.DataFrame(iv_results)

def iv_interpretation(iv):
    if pd.isna(iv): return "Error"
    if iv < 0.02: return 'Inútil'
    elif iv < 0.1: return 'Débil'
    elif iv < 0.3: return 'Medio'
    elif iv < 0.5: return 'Fuerte'
    else: return 'Sospechoso'
df_iv_all['Interpretation'] = df_iv_all['IV_Score'].apply(iv_interpretation)
print("\nCálculo de IV completado para todas las variables.")

# --- 3. Seleccionar Candidatas y Preparar Análisis de Correlación ---
candidate_features = df_iv_all[
    (df_iv_all['IV_Score'] >= 0.02) & 
    (df_iv_all['Interpretation'] != 'Sospechoso')
]['Variable'].tolist()

print(f"\nSe han seleccionado {len(candidate_features)} variables candidatas con IV >= 0.02.")
iv_lookup = df_iv_all.set_index('Variable')['IV_Score'].to_dict()

features_in_df = [f for f in candidate_features if f in df_model.columns]
df_candidates = df_model[features_in_df]
numeric_features_for_corr = df_candidates.select_dtypes(include=np.number).columns.tolist()

# --- 4. Identificar y Resolver Conflictos de Correlación ---
corr_matrix = df_model[numeric_features_for_corr].corr().abs()
upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# --- INICIO DE LA MODIFICACIÓN ---
conflict_summary = []
to_drop = set()

for column in upper_tri.columns:
    high_corr_features = upper_tri[column][upper_tri[column] > CORRELATION_THRESHOLD]
    for feature, corr_value in high_corr_features.items():
        var1, var2 = column, feature
        iv1, iv2 = iv_lookup.get(var1, 0), iv_lookup.get(var2, 0)
        
        # Decidir cuál variable descartar
        if iv1 < iv2:
            var_to_drop = var1
            var_to_keep = var2
        else:
            var_to_drop = var2
            var_to_keep = var1
            
        to_drop.add(var_to_drop)
        
        # Añadir detalles del conflicto al resumen
        conflict_summary.append({
            'Variable_1': var1,
            'Variable_2': var2,
            'Correlation': corr_value,
            'IV_Variable_1': iv1,
            'IV_Variable_2': iv2,
            'Variable_To_Drop': var_to_drop,
            'Variable_To_Keep': var_to_keep
        })

# --- 5. Presentar Resultados en un DataFrame ---
print(f"\n--- Resumen del Análisis de Multicolinealidad (Umbral > {CORRELATION_THRESHOLD}) ---")

if conflict_summary:
    df_correlation_summary = pd.DataFrame(conflict_summary)
    df_correlation_summary = df_correlation_summary.sort_values(by='Correlation', ascending=False).reset_index(drop=True)
    
    print(f"Se encontraron {len(df_correlation_summary)} conflictos de alta correlación.")
    # Mostrar el DataFrame de resumen
    display(df_correlation_summary)

    final_drop_list = sorted(list(to_drop))
    print(f"\nSe recomienda eliminar las siguientes {len(final_drop_list)} variables:")
    print(final_drop_list)
    
    final_features = [f for f in candidate_features if f not in final_drop_list]
    print(f"\nEl número final de variables recomendadas es: {len(final_features)}")
else:
    print("¡Excelente! No se encontraron variables con correlación por encima del umbral.")
    final_features = candidate_features
    print(f"El número final de variables se mantiene en: {len(final_features)}")

# --- FIN DE LA MODIFICACIÓN ---

In [33]:
list_corr_features_drop = list(df_correlation_summary['Variable_To_Drop'].unique())
print('Dimension of features to drop:', len(list_corr_features_drop))

df_model_f = df_model.drop(list_corr_features_drop, axis=1)

print('Final Dimension of Dataset:', df_model_f.shape)

Dimension of features to drop: 15
Final Dimension of Dataset: (1348059, 45)


In [35]:
df_iv_all[df_iv_all['Variable'].isin(df_model_f.columns)].sort_values(by='IV_Score', ascending=False).reset_index(drop=True).head(50)

,Variable,IV_Score,Interpretation
0,emp_title,6.741013,Sospechoso
1,last_pymnt_amnt,4.177115,Sospechoso
2,last_fico_range_low,4.098737,Sospechoso
3,last_fico_range_high,4.098737,Sospechoso
4,title,1.043574,Sospechoso
5,sub_grade,0.495516,Fuerte
6,grade,0.461092,Fuerte
7,int_rate,0.449055,Fuerte
8,term,0.173626,Medio
9,fico_range_high,0.122664,Medio


### Saving

In [ ]:
####Savind data until here

df_model_f = df_model_f.drop(['last_pymnt_amnt', 'last_fico_range_low', 'last_fico_range_high' ], axis=1)


df_model_f.reset_index(drop=True).to_csv("df_model_to_fe.csv", index=False)

In [38]:
df_model_f.shape

(1348059, 42)